In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import re
dataset=pd.read_csv("/kaggle/input/train-news/train.csv")
dataset.columns=['id','content','comment_all','label']
dataset=dataset.set_index('id')
test=pd.read_csv("/kaggle/input/train-news/test.csv")
test.columns=['id','content','comment_all']
test=test.set_index('id')
testset=test


In [5]:
import jieba
def clear(text)->str:
    #print(re.findall('[\u4E00-\u9FFF]+|\d',text))
    if type(text) is float:return 
    str1=""
    str2=str1.join(text)
    y=lambda x:re.findall('[\u4E00-\u9FFF]+',x)
    li=y(str2)
    for i in range(len(li)):
        li[i]=list(jieba.cut(li[i]))
    li2=[]
    for subli in li:
        for item in subli:
            li2.append(item)
    return li2
text='正则365表达\t式ds匹配/--*测\t试文字'
a=clear(dataset.loc[9982,'content'])
b=clear(dataset.loc[9984,'comment_all'])
c=clear(testset.loc[9979,'content'])
for index,row in dataset.iterrows():
    dataset.at[index,'content']=clear(dataset.at[index,'content'])
    dataset.at[index,'comment_all']=clear(dataset.at[index,'comment_all'])




In [6]:
dataset

In [7]:
dataset2=[]
for index,row in dataset.iterrows():
    str2=" "
    if dataset.loc[index,'comment_all']is None:
        dataset2.append(dataset.loc[index,'content'])
    else:
        dataset2.append(dataset.loc[index,'content']+dataset.loc[index,'comment_all'])
for index,row in testset.iterrows():
    testset.at[index,'content']=clear(testset.at[index,'content'])
    testset.at[index,'comment_all']=clear(testset.at[index,'comment_all'])

In [8]:
dataset.to_pickle('split_dataset')
testset.to_pickle('split_testset')

In [9]:
import pickle
dataset=open('split_dataset','rb')
dataset=pickle.load(dataset)
testset=open('split_testset','rb')
testset=pickle.load(testset)

In [10]:
def label_get(dataset):
    labelset=[]
    for index,row in dataset.iterrows():
        labelset.append(dataset.loc[index,'label'])
    return labelset

labelset=label_get(dataset)
def pd_li_transform(dataset):
    dataset_list=[]
    for index,row in dataset.iterrows():
        str2=" "
        if dataset.loc[index,'comment_all']is None:
            dataset_list.append(dataset.loc[index,'content'])
        else:
            dataset_list.append(dataset.loc[index,'content']+dataset.loc[index,'comment_all'])
    return dataset_list
testset_list=pd_li_transform(testset)
dataset_list=pd_li_transform(dataset)
def list_deal(dataset,labelset):
    labelset2=[]
    dataset2=[]
    for i in range(len(dataset)):
        if (dataset[i]!=[]):
            dataset2.append(dataset[i])
            labelset2.append(labelset[i])
    return dataset2,labelset2
dataset_list,labelset=list_deal(dataset_list,labelset)
def list_onehot(labelset):
    newli=[]
    for index in labelset:
        if (index==-1):newli.append([-1.0,0.0,0.0])
        if (index==0):newli.append([0.0,-1.0,0.0])
        if (index==1):newli.append([0.0,0.0,-1.0])
    return newli
def list_float(labelset):
    newli=[]
    for index in labelset:
        if (index==-1):newli.append(0.0)
        if (index==0):newli.append(1.0)
        if (index==1):newli.append(2.0)
    return newli
labelset3=list_float(labelset)
labelset2=list_onehot(labelset)
def li_str_transform(dataset_list):
    dataset2=[]
    for seq in dataset_list:
        str1=' '.join(seq)
        dataset2.append(str1)
    return dataset2
dataset_str=li_str_transform(dataset_list)
testset_str=li_str_transform(testset_list)


In [14]:
from fastNLP import DataSet
data_init=DataSet({'raw_chars':dataset_list,'target':labelset})
data_train=data_init[:30000]
data_validation=data_init[30001:35000]
data_test=DataSet({'raw_chars':testset_list})
from fastNLP import Vocabulary
vocab=Vocabulary()
vocab.from_dataset(data_train,field_name='raw_chars',no_create_entry_dataset=[data_validation])
vocab.index_dataset(data_train,field_name='raw_chars')
vocab.index_dataset(data_validation,field_name='raw_chars')
vocab.index_dataset(data_test,field_name='raw_chars')
target_vocab=Vocabulary(padding=None,unknown=None)
target_vocab.from_dataset(data_train,field_name='target')
target_vocab.index_dataset(data_train,field_name='target')
target_vocab.index_dataset(data_validation,field_name='target')

In [12]:
pip install fastnlp

In [ ]:
data_train

In [15]:
data_train.rename_field('raw_chars','words')
data_validation.rename_field('raw_chars','words')
data_train.set_input('words')
data_train.set_target('target')
data_validation.set_input('words')
data_validation.set_target('target')


In [ ]:
print(len(data_test))

In [17]:
import torch
from fastNLP.embeddings import BertEmbedding
from fastNLP import Vocabulary
embed=BertEmbedding(vocab,model_dir_or_name='cn-wwm', include_cls_sep=True,auto_truncate=True)
from fastNLP.models import BertForSequenceClassification
model = BertForSequenceClassification(embed,num_labels=3)

In [19]:
from fastNLP import Trainer, CrossEntropyLoss, AccuracyMetric, Adam
device = 0 if torch.cuda.is_available() else 'cpu'
print(device)
trainer = Trainer(data_train, model,
                  optimizer=Adam(model_params=model.parameters(), lr=2e-5),
                  loss=CrossEntropyLoss(), device=device,
                  batch_size=8,dev_data=data_validation,
                  metrics=AccuracyMetric(), n_epochs=5, print_every=1)
trainer.train()

In [21]:
def batch_predict(model,data,device):
    submission=pd.DataFrame(columns=['Prediction'])
#    submission = pd.DataFrame(columns=['ID','Prediction'])
    submission=[]
    for i in range(len(data)):
    #for i in range(5):
#         print(data.words[i])
        if (i%100==0):print(f"第{i}次预测完成。")
        model=model.to(device)
        tensor = torch.tensor(data['raw_chars'][i]).to(device)
        pred = model.predict(tensor.view(1,-1))
        prob = pred['pred'].cpu().numpy()[0]
        submission.append(prob)
    return submission
submission=batch_predict(model,data_test,device)


In [23]:
submission2=[]
for i in range(len(submission)):
    if(submission[i]==2):submission2.append(0)
    elif(submission[i]==0):submission2.append(1)
    elif(submission[i]==1):submission2.append(-1)
    else:submission2.append(submission[i])

np.savetxt('/kaggle/working/bert.txt',submission2,fmt='%d',delimiter='\n')
print("OK")